In airbus ship detection competition, all ships are depicted as rotated rectagles. Here is a code to get a rotated rectangle from a mask,  which gives length of major axis (from bow to stern),minor axis(width), center position, and rotation from x-axis.

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import PIL.ImageDraw as ImageDraw, PIL.Image as Image

In [ ]:
def rle_decode(rle):
    if not isinstance(rle, str):
        return None
    
    s = np.array([int(w) for w in rle.split()]).reshape(-1, 2)
    s[:, 0] -= 1  # offset from 0
    mask = np.zeros(768*768, dtype=np.uint8)
    for start, length in s:
        mask[start:start+length] = 255
    return mask.reshape(768, 768).T

def boundbox(mask):
    nzx = np.nonzero(mask.sum(axis=0))[0]
    x0, x1 = nzx[0], nzx[-1] + 1

    nzy = np.nonzero(mask[:, x0:x1].sum(axis=1))[0]
    y0, y1 = nzy[0], nzy[-1] + 1
    
    return [x0, y0, x1, y1]

def bb_union(bb1, bb2):
    return [min(bb1[0], bb2[0]), min(bb1[1], bb2[1]), max(bb1[2], bb2[2]), max(bb1[3], bb2[3])]

def get_mask(L, W, ang, xc=None, yc=None, mask_reuse=None):
    L2, W2 = L / 2, W / 2

    pts = np.array([[-L2,-W2], [L2,-W2], [L2,W2], [-L2,W2]])

    c, s = np.cos(ang * np.pi / 180), np.sin(ang * np.pi / 180)
    a = np.array([[c, s], [-s, c]])
    pts = np.dot(pts, a)
    
    xmin, ymin = pts.min(axis=0)
    xmax, ymax = pts.max(axis=0)
    sx, sy = np.int((xmax - xmin) * 1.1), np.int((ymax - ymin) * 1.1)
    if xc is None: 
        w = sx
        xc = w / 2
    else:
        w = xc + sx
        w = np.clip(w, 0, 768)
        
    if yc is None: 
        h = np.clip(sy, None, 768)
        yc = h / 2
    else:
        h = yc + sy
        h = np.clip(h, None, 768)
        
    dx, dy = xc - (xmax + xmin)/2, yc - (ymax + ymin)/2
    
    if mask_reuse is not None:
        if (mask_reuse.shape[0] < h) or (mask_reuse.shape[1] < w):
            print(mask_reuse.shape, h, w)
            mask_reuse = None
        else:
            mask_reuse.fill(0)
        
    img = Image.fromarray(mask_reuse) if mask_reuse is not None else Image.new('L', (int(w), int(h)))
    drw = ImageDraw.Draw(img)
    
    pts1 = pts + [dx,dy]
    drw.polygon([ (round(x),round(y)) for x,y in pts1.tolist() ], fill=255)
    
    return np.array(img)

def mask_to_rectangle(mask):
    # get rectangle from mask using eigensolutions 
    # returns major-axis-len, minor-axis-len, angle, x-center, y-center
    h, w = mask.shape
    xa, ya = np.meshgrid(np.arange(0, w), np.arange(0, h))
    ii, xa, ya = mask.flatten() > 0, xa.flatten(), ya.flatten()
    xa, ya = xa[ii], ya[ii]
    xc, yc = xa.mean(), ya.mean()
    a11, a12, a22 = np.mean(xa * xa) - xc * xc, np.mean(xa * ya) - xc * yc, np.mean(ya * ya) - yc * yc
    b = -a11 - a22
    c = a11 * a22 - a12 * a12
    d = np.sqrt(b * b - 4 * c)
    l1, l2 = (-b + d) / 2, (-b - d) / 2
    fac = np.sqrt(3) * 2
    L1, W1 = np.sqrt(l1) * fac, np.sqrt(l2) * fac
    ang1 = np.arctan2(l1 - a11, a12) * 180 / np.pi
    return [L1, W1, ang1, xc, yc]


In [ ]:
!ls ../input

In [ ]:
dftrain = pd.read_csv('../input/train_ship_segmentations.csv')
rles = dftrain[dftrain.EncodedPixels.notnull()].EncodedPixels.values

In [ ]:
rle = np.random.choice(rles)
mask = rle_decode(rle)
mask_param = mask_to_rectangle(mask)
print(mask_param)
mask1 = get_mask(*mask_param, np.zeros_like(mask))

bb = bb_union(boundbox(mask), boundbox(mask1))
print(bb)
mask = mask[bb[1]:bb[3], bb[0]:bb[2]]
mask1 = mask1[bb[1]:bb[3], bb[0]:bb[2]]

plt.figure(figsize=(8, 8))
plt.imshow(mask, cmap='Reds', alpha=0.7)
plt.imshow(mask1, cmap='Blues', alpha=0.3)
plt.show()

mask, mask1 = mask.astype(np.int32), mask1.astype(np.int32)
iou = np.sum(mask * mask1 > 0) / (np.sum(mask + mask1 > 0)) 
print(iou)